In [ ]:
from fh_id_data import fh_id_data
from fh_ca import campo_amostral
from fh_filter import get_filter
import os
from supabase import create_client, Client
import math
table = 'table_statics'
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)


In [ ]:
def mount_gols_filters(fh_q_dict, dict_campo_amostral):
    category = 'gols'
    gol_atual = fh_q_dict['gols']['sum_period1']
    ca_category = dict_campo_amostral['gols'] 
    print(gol_atual)
    dict_gols_filter_over = {
        1 :  [gol_atual + 0.5, get_filter(fh_q_dict, category, 1),0,0],
        2 : [gol_atual + 1.5, get_filter(fh_q_dict, category, 2), 0,0],
        3 : [gol_atual + 2.5, get_filter(fh_q_dict, category, 3), 0,0]
    }

    dict_gols_filter_under = {
        1 : [gol_atual + 0.5 ,ca_category - get_filter(fh_q_dict, category, 1), 0, 0],
        2 : [gol_atual + 1.5, ca_category - get_filter(fh_q_dict, category, 2),0, 0],
        3 : [gol_atual + 2.5, ca_category - get_filter(fh_q_dict, category, 3),0, 0]
    }

    return dict_gols_filter_over, dict_gols_filter_under


def mount_gols_resp(dict_gols_filter_over, dict_gols_filter_under):
    for i in range(1,len(dict_gols_filter_over) + 1):
        prob = (dict_gols_filter_over[i][1] * 100) / dict_campo_amostral['gols']
        dict_gols_filter_over[i][2] = prob
        odd = 1/(prob/100)
        dict_gols_filter_over[i][3] = odd

    for i in range(1,len(dict_gols_filter_under) + 1):
        prob = (dict_gols_filter_under[i][1] * 100) / dict_campo_amostral['gols']
        dict_gols_filter_under[i][2] = prob
        odd = 1/(prob/100)
        dict_gols_filter_under[i][3] = odd

    return dict_gols_filter_over, dict_gols_filter_under




In [ ]:
def mount_corners_filters(fh_q_dict, dict_campo_amostral):
    category = 'corners'
    corners_atual = fh_q_dict['corners']['sum_1st_cornerkicks'][2]
    ca_category = dict_campo_amostral['corners'] 
    print(corners_atual)
    dict_corners_filter_over = {
        1 : [corners_atual + 3, get_filter(fh_q_dict, category, 3),0,0],
        2 : [corners_atual + 4, get_filter(fh_q_dict, category, 4), 0,0],
        3 : [corners_atual + 5, get_filter(fh_q_dict, category, 5), 0,0],
        4 : [corners_atual + 6, get_filter(fh_q_dict, category, 6), 0,0],
        5 : [corners_atual + 7, get_filter(fh_q_dict, category, 7), 0,0],
    }

    dict_corners_filter_under = {
        1 : [corners_atual + 3, ca_category - get_filter(fh_q_dict, category, 3),0, 0],
        2 : [corners_atual + 4, ca_category - get_filter(fh_q_dict, category, 4),0, 0],
        3 : [corners_atual + 5, ca_category - get_filter(fh_q_dict, category, 5),0, 0],
        4 : [corners_atual + 6, ca_category - get_filter(fh_q_dict, category, 6),0, 0],
        5 : [corners_atual + 7, ca_category - get_filter(fh_q_dict, category, 7),0, 0],
    }

    return dict_corners_filter_over, dict_corners_filter_under



def mount_corners_resp(dict_corners_filter_over, dict_corners_filter_under):
    for i in range(1,len(dict_corners_filter_over) + 1):
        prob = (dict_corners_filter_over[i][1] * 100) / dict_campo_amostral['gols']
        dict_corners_filter_over[i][2] = prob
        odd = 1/(prob/100)
        dict_corners_filter_over[i][3] = odd

    for i in range(1,len(dict_corners_filter_under) + 1):
        prob = (dict_corners_filter_under[i][1] * 100) / dict_campo_amostral['gols']
        dict_corners_filter_under[i][2] = prob
        if prob != 0:
            odd = 1/(prob/100)
        elif prob == 0:
            odd = 10
        dict_corners_filter_under[i][3] = odd

    return dict_corners_filter_over, dict_corners_filter_under


In [66]:
event_id = 13472618
dict_campo_amostral, fh_q_dict = campo_amostral(event_id)
dict_campo_amostral, fh_q_dict

({'gols': 116, 'corners': 142, 'yellowcards': 574},
 {'gols': {'sum_period1': 0,
   'sum_expected_goals_1st': (0.55, 1.05),
   'sum_1st_totalshotsongoal': (11, 15),
   'sum_1st_shotsongoal': (3, 5),
   'sum_1st_totalshotsinsidebox': (4, 6),
   'sum_1st_finalthirdentries': (43, 73),
   'sum_1st_cornerkicks': (3, 7, 5),
   'sum_1st_freekicks': (10, 14),
   'sum_1st_yellowcards': (1, 3, 2)},
  'corners': {'sum_period1': 0,
   'sum_expected_goals_1st': (0.55, 1.05),
   'sum_1st_totalshotsongoal': (11, 15),
   'sum_1st_shotsongoal': (3, 5),
   'sum_1st_totalshotsinsidebox': (4, 6),
   'sum_1st_finalthirdentries': (43, 73),
   'sum_1st_cornerkicks': (3, 7, 5),
   'sum_1st_freekicks': (10, 14),
   'sum_1st_yellowcards': (1, 3, 2)},
  'yellowcards': {'sum_period1': 0,
   'sum_expected_goals_1st': (0.55, 1.05),
   'sum_1st_totalshotsongoal': (11, 15),
   'sum_1st_shotsongoal': (3, 5),
   'sum_1st_totalshotsinsidebox': (4, 6),
   'sum_1st_finalthirdentries': (43, 73),
   'sum_1st_cornerkicks': (

In [67]:
dict_corners_filter_over, dict_corners_filter_under = mount_corners_filters(fh_q_dict, dict_campo_amostral)
mount_corners_resp(dict_corners_filter_over, dict_corners_filter_under)

5


({1: [8, 128, 110.34482758620689, 0.90625],
  2: [9, 111, 95.6896551724138, 1.045045045045045],
  3: [10, 93, 80.17241379310344, 1.2473118279569892],
  4: [11, 67, 57.758620689655174, 1.7313432835820897],
  5: [12, 47, 40.51724137931034, 2.4680851063829787]},
 {1: [8, 14, 12.068965517241379, 8.285714285714286],
  2: [9, 31, 26.724137931034484, 3.741935483870967],
  3: [10, 49, 42.241379310344826, 2.3673469387755106],
  4: [11, 75, 64.65517241379311, 1.5466666666666666],
  5: [12, 95, 81.89655172413794, 1.2210526315789474]})

In [68]:
dict_gols_filter_over, dict_gols_filter_under = mount_gols_filters(fh_q_dict, dict_campo_amostral)
mount_gols_resp(dict_gols_filter_over, dict_gols_filter_under)

0


({1: [0.5, 95, 81.89655172413794, 1.2210526315789474],
  2: [1.5, 60, 51.724137931034484, 1.9333333333333331],
  3: [2.5, 24, 20.689655172413794, 4.833333333333333]},
 {1: [0.5, 21, 18.103448275862068, 5.523809523809525],
  2: [1.5, 56, 48.275862068965516, 2.0714285714285716],
  3: [2.5, 92, 79.3103448275862, 1.2608695652173914]})